# 0.0 Imports

## 0.1. Libries

In [1]:
import pandas as pd

from sklearn.model_selection         import train_test_split
from sklearn.linear_model            import LogisticRegression as lr
from sklearn.feature_extraction.text import CountVectorizer

## 0.2. Load Data

In [2]:
df_raw = pd.read_csv('data/imdb-reviews-pt-br.csv')

## 0.3 Helper Functions

In [3]:
def classificar_texto(df, coluna_texto, target, n_features):

    #Modelo do tipo bag_of_words usando CountVectorizer
    vetorizar = CountVectorizer(lowercase=False, max_features = n_features)#max_features delimite o tamanho do vetor usado
    bag_of_words = vetorizar.fit_transform(df[coluna_texto])

    #separa treino e teste
    treino, teste, classse_treino, classe_teste = train_test_split(bag_of_words,
                                                                  df[target], 
                                                                  random_state = 42)
    #treinar modelo
    regressao_logistica = lr(solver = 'lbfgs')
    regressao_logistica.fit(treino, classe_treino)
    return regressao_logistica.score( teste, classe_teste )

# 1.0. Data Description

In [4]:
df1 = df_raw.copy()

## 1.1. Data Dimensions

In [5]:
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Number of cols: {}'.format( df1.shape[1] ) )

Number of Rows: 49459
Number of cols: 4


## 1.2. Data Types

In [6]:
df1.dtypes

id            int64
text_en      object
text_pt      object
sentiment    object
dtype: object

## 1.3. Check NA

In [7]:
df1.isna().sum()

id           0
text_en      0
text_pt      0
sentiment    0
dtype: int64

# 2.0 Feature Engineering

In [8]:
df2 = df1.copy()

In [9]:
#coluna classificação, transformar valores de sentiment para binario
classificacao = df2['sentiment'].replace(['neg', 'pos'], [0,1])
df2['classificacao'] = classificacao

# 3.0 EAD

In [10]:
df3 = df2.copy()

In [11]:
df3.sample(10)

,id,text_en,text_pt,sentiment,classificacao
25590,25592,The saddest part of this is the fact that thes...,A parte mais triste disso é o fato de que esse...,neg,0
7616,7618,"First of all, Id like to say I am a teenager s...","Primeiro de tudo, eu gostaria de dizer que eu ...",neg,0
23633,23635,I wanted to see this movie ever since the prev...,Eu queria ver esse filme desde que as prévias ...,pos,1
17936,17938,This is an entertaining surreal road movie. It...,Este é um divertido filme de estrada surreal. ...,pos,1
12564,12566,Dr. Ben McKenna James Stewart and Jo McKenna D...,Dr. Ben McKenna James Stewart e Jo McKenna Dor...,pos,1
34531,34533,Bridges of madison county is a better made ver...,Bridges of madison county é uma versão melhora...,neg,0
33880,33882,God this film was just so boring apart from th...,"Deus, este filme foi tão chato, além da música...",neg,0
34444,34446,It definitely fits the time period as the Axis...,Isso definitivamente se encaixa no período de ...,neg,0
39674,39676,"""Bread"" very sharply skewers the conventions o...","""Pão"" espeta muito agudamente as convenções de...",pos,1
9225,9227,Mr. Brento wonders if this movie was produced ...,Sr. Brento se pergunta se este filme foi produ...,neg,0


## 3.1. Analisando text_pt

> Aqui vamos dar uma olhada em uma resenha que seja positiva e uma negativa, para ver os tipos de palavras usadas e pensar como o algortmo poderia reconhecer uma resenha como positiva ou negativa. 

In [12]:
#exemplo de resenha negativa
print('Negativa \n')
print(df3.text_pt[189])

Negativa 

Este é sem dúvida o pior filme que eu já vi. E acredite em mim, eu vi muitos filmes. A reviravolta inacreditável que o filme faz - passando de um extremamente mau filme "Formas de vida alienígenas habitam a terra", com um filme que tenta espalhar um arquicristiano "O dia do julgamento está próximo, buscar Jesus ou queimar por toda a eternidade em as dívidas ardentes do inferno "mensagem - deixou-me atordoado depois de ter sido atormentado por 85 minutos. Até mesmo os cristãos religiosos devem se envergonhar ou ficar furiosos ao ver suas crenças postadas dessa maneira. Eu não sabia o que fazer comigo quando assisti a atuação horrível que poderia ter sido realizada por crianças de 7 anos de idade. Simplesmente repugnante. Eu não sou cristão nem muito religioso. Mas se eu estivesse, não teria mais medo do Inferno. Rich Christiano mostrou ser algo muito pior.


In [13]:
#exemplo de resenha positiva
print('Positiva \n')
print(df3.text_pt[49002])

Positiva 

Crescendo em Nova York no final dos anos 80 e início dos anos 90, posso dizer pessoalmente que este é um dos documentários mais importantes feitos para cobrir esse lugar neste período de tempo. Não Madonna não veio com a idéia de Voguing, mas é de onde ela tirou! Em vez de combater a violência uns dos outros ou em brigas de gato, o voguing permitia que as pessoas "lutassem" dentro dos confins de tudo, menos que tocassem umas às outras, o que justificaria uma desqualificação automática. Vendo este tipo de extraordinariamente talentosas / bem orquestradas "jogadas" nos clubes foi nada menos do que espetacular e todos os grandes nomes de antigamente estão aqui ... Pepper La Beija, Paris Duprée, Xtragavaganza, etc. ..tudo comemorado nos gostos de peças de época como a música de Malcom McLarens "Deep in Vogue" ... não importava quem você era, ou de onde você era porque quando você passava por aquelas portas nesse "reino mágico" de De certa forma, você se tornou parte de algo maio

## 3.2. Analisando variável alvo, classificacao

In [14]:
#Analisando o balanceamento
print(round(df3.classificacao.value_counts(normalize=True)*100, 2) )

0    50.07
1    49.93
Name: classificacao, dtype: float64


> Temos a variável alvo bem balanceada, com praticamente metade indicando respostas positivas e metade respostas negativas. 

# 4.0 Data Preparation

In [15]:
df4 = df3.copy()

In [16]:
#Separar dados em treino e teste
treino, teste, classe_treino, classe_teste = train_test_split(df4.text_pt, 
                                                              df4.classificacao,
                                                              random_state = 42)

# 5.0 Machine Learning

In [17]:
df5 = df4.copy()

##  LogisticRegression - Bag of Words 

>Usaremos o LogisticRegression como primeiro modelo para classificar os textos como boa avaliação ou ruim. 

> Para usarmos o LogisticRegression é necessário usar uma técnica de vetorização dos textos, foi usado o bag of words. 

In [18]:
acuracia = classificar_texto(df5, 'text_pt', 'classificacao', 200 )
print('Acuracia medida pelo score da regressao logistica é: ', acuracia)


Acuracia medida pelo score da regressao logistica é:  0.7534169025475131


> Neste primeiro ciclo, usando uma configuração de 200 para o tamanho da matriz do modelo de LogistcRegression, conseguimos obter uma acertividade de 75,34%